In [2]:
import pandas as pd
import os
from pathlib import Path


In [3]:
def format_tuple(x):
    try:
        return eval(x)[0]
    except:
        return None

In [4]:
from gama import GamaClassifier
from gama.genetic_programming.components import Individual
from sklearn.impute import SimpleImputer

clf = GamaClassifier(max_total_time=3, store = 'nothing')

def convert_str_to_pipeline(pipeline_str: str):
    
    individual = Individual.from_string(
        pipeline_str,
        clf._pset,
        clf._operator_set._compile
    )
    pipeline = individual.pipeline
    pipeline.steps.insert(0, ('imputation', SimpleImputer(strategy='median')))
    pipeline_as_str = f"{pipeline}".replace(" ", "").replace("\n", "")

    return pipeline_as_str

In [5]:
def pipeline_str_to_sklearn_format(pipeline_str: str):
    if pipeline_str.startswith("Pipeline"):
        return pipeline_str
    else:
        return convert_str_to_pipeline(pipeline_str)
    

In [7]:
log_path = Path(parent_dir + "/src/logs/multiclass")
candidates = set()

for folder in log_path.iterdir():

    #get the digits from the folder name at the end
    dataset_id = str(folder).split("_")[-1]
    current_log = pd.read_csv(f"{folder}/evaluations.log", delimiter = ';')
    current_log["pipeline"] = current_log["pipeline"].apply(lambda x: pipeline_str_to_sklearn_format(x))
    current_log.drop_duplicates(subset = "pipeline", inplace = True)
    current_log["score"] = current_log["score"].apply(lambda x: format_tuple(x)).astype(float)
    top_ten = current_log.nlargest(10, "score", keep = "first")["pipeline"].to_list()
    counter = 0
    for item in top_ten:
        if item not in candidates:
            candidates.add(item)
            counter += 1
        if counter == 5:
            break
        if item == top_ten[-1]:
            print(f"Could not find 5 unique pipelines for dataset {dataset_id}")


FileNotFoundError: [Errno 2] No such file or directory: '/home/wichert/Master DS&AI/master_thesis/src/logs/binary'